In [1]:
# Import dependencies
import sqlite3
import pandas as pd
import numpy as np
from numpy import nan

In [2]:
# Connect to database
conn = sqlite3.connect('database.sqlite')

In [3]:
# Read sqlite query results into a pandas DataFrame
scholl_df = pd.read_sql_query('SELECT * from schools_table order by School_name',conn)
scholl_df

,index,school_id,school_name,type,size,budget
0,7,7,Farmer High School,District,3429,2235708
1,9,9,Floyd High School,Charter,2104,1224528
2,8,8,Greene High School,District,4690,3062570
3,2,2,Hood High School,Charter,930,578460
4,1,1,Long High School,Charter,628,368636
5,3,3,Lopez High School,District,3428,2248768
6,10,10,Patterson High School,District,4389,2887962
7,0,0,Stewart High School,Charter,1208,740504
8,5,5,Thompson High School,Charter,1353,838860
9,4,4,Vargas High School,Charter,2479,1574165


In [4]:
# Read sqlite query results into a pandas DataFrame
student_df = pd.read_sql_query('SELECT * from students_table',conn)
student_df.head()

,index,student_id,student_name,gender,grade,school_name,reading_score,math_score
0,0,0,Kevin Logan,M,9th,Stewart High School,93,98
1,1,1,Earl Bush,M,10th,Stewart High School,99,92
2,2,2,Jeffrey Williamson,M,10th,Stewart High School,98,98
3,3,3,Karen Dean,F,10th,Stewart High School,97,77
4,4,4,Rebecca Reyes,F,11th,Stewart High School,89,87


# District Summary

In [5]:
# Read sqlite query results into a pandas DataFrame
district_summary = pd.read_sql_query("""

SELECT count(DISTINCT(school_id)) as Total_Schools,
count(student_id) as Total_Students,
sum(budget) as Total_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
math_passing_rate,
reading_passing_rate,
(math_passing_rate + reading_passing_rate)/2 as Overall_Passing_Rate

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

join

(select
cast(sum(mathover70) as float)/cast(count(mathover70) as float) as Math_Passing_Rate
from(
select
 student_id
 ,math_score
 , case when math_score > 70 then 1
		else 0
end as mathover70
from 
students_table))

join

(select
cast(sum(readingover70) as float)/cast(count(readingover70) as float) as Reading_Passing_Rate
from(
select
student_id ,reading_score, 
case when reading_score > 70 then 1
		else 0
end as readingover70
from 
students_table));

""",conn)

district_summary.head()

,Total_Schools,Total_Students,Total_Budget,Average_Math_Score,Average_Reading_Score,Math_Passing_Rate,Reading_Passing_Rate,Overall_Passing_Rate
0,11,27712,57663546353,82.164802,82.190279,0.836894,0.751876,0.794385


#  School Summary

In [6]:
# Read sqlite query results into a pandas DataFrame
school_summary = pd.read_sql_query("""

SELECT 
c.School_name,
c.School_type,
c.Total_Budget,
c.Total_Student,
c.Per_Student_Budget,
c.Average_Math_Score,
c.Average_Reading_Score,
c.Average_Overall_Score,
(pass_count*100 / all_count) ||'%' as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_Type,
sum(budget) as Total_Budget,
count(student_id) as Total_Student,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)
group by School_name
order by Per_Student_Budget
;

""",conn)

school_summary

,School_name,School_Type,Total_Budget,Total_Student,Per_Student_Budget,Average_Math_Score,Average_Reading_Score,Average_Overall_Score,Overall_Passing_Rate
0,Long High School,Charter,231503408,628,368636,83.068471,93.810510,88.439490,89%
1,Hood High School,Charter,537967800,930,578460,83.574194,94.077419,88.825806,89%
2,Stewart High School,Charter,894528832,1208,740504,83.774007,94.120033,88.947020,91%
3,Thompson High School,Charter,1134977580,1353,838860,83.515891,94.115299,88.815595,91%
4,Floyd High School,Charter,2576406912,2104,1224528,83.028042,93.966255,88.497148,90%
5,Vargas High School,Charter,3902355035,2479,1574165,83.576442,93.961678,88.769060,90%
6,Webb High School,District,5972068832,3074,1942768,81.415420,76.558230,78.986825,49%
7,Farmer High School,District,7666242732,3429,2235708,81.594051,77.017498,79.305774,52%
8,Lopez High School,District,7708776704,3428,2248768,81.725788,76.875146,79.300467,51%
9,Patterson High School,District,12675265218,4389,2887962,81.481431,76.644566,79.062998,50%


# Top Performing Schools (By Passing Rate)

In [7]:
# Read sqlite query results into a pandas DataFrame
top5 = pd.read_sql_query("""

SELECT 
c.School_name,
c.School_type,
c.Total_Budget,
c.Per_Student_Budget,
c.Average_Math_Score,
c.Average_Reading_Score,
c.Average_Overall_Score,
(pass_count*100 / all_count) ||'%' as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_type,
sum(budget) as Total_Budget,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)
group by School_name
order by c.Average_Overall_Score
limit 5
;

""",conn)

top5

,School_name,School_type,Total_Budget,Per_Student_Budget,Average_Math_Score,Average_Reading_Score,Average_Overall_Score,Overall_Passing_Rate
0,Webb High School,District,5972068832,1942768,81.415420,76.558230,78.986825,49%
1,Patterson High School,District,12675265218,2887962,81.481431,76.644566,79.062998,50%
2,Greene High School,District,14363453300,3062570,81.695522,76.807463,79.251493,51%
3,Lopez High School,District,7708776704,2248768,81.725788,76.875146,79.300467,51%
4,Farmer High School,District,7666242732,2235708,81.594051,77.017498,79.305774,52%


# Bottom Performing Schools (By Passing Rate)

In [8]:
# Read sqlite query results into a pandas DataFrame
bottom5 = pd.read_sql_query("""

SELECT 
c.School_name,
c.School_type,
c.Total_Budget,
c.Per_Student_Budget,
c.Average_Math_Score,
c.Average_Reading_Score,
c.Average_Overall_Score,
(pass_count*100 / all_count) ||'%' as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_type,
sum(budget) as Total_Budget,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)
group by School_name
order by c.Average_Overall_Score desc
limit 5
;

""",conn)

bottom5

,School_name,School_type,Total_Budget,Per_Student_Budget,Average_Math_Score,Average_Reading_Score,Average_Overall_Score,Overall_Passing_Rate
0,Stewart High School,Charter,894528832,740504,83.774007,94.120033,88.947020,91%
1,Hood High School,Charter,537967800,578460,83.574194,94.077419,88.825806,89%
2,Thompson High School,Charter,1134977580,838860,83.515891,94.115299,88.815595,91%
3,Vargas High School,Charter,3902355035,1574165,83.576442,93.961678,88.769060,90%
4,Floyd High School,Charter,2576406912,1224528,83.028042,93.966255,88.497148,90%


# Math Scores by Grade

In [9]:
# Read sqlite query results into a pandas DataFrame
math_score_by_grade = pd.read_sql_query("""

select 

*

from 

(SELECT 

school_name,
avg(math_score) as '9th'

from 

schools_table a

join

students_table b

using (school_name)

where grade = '9th'

group by school_name) c

join 

(SELECT 

school_name,
avg(math_score) as '10th'

from 

schools_table a

join

students_table b

using (school_name)

where grade = '10th'

group by school_name)

using (school_name)

join 

(SELECT 

school_name,
avg(math_score) as '11th'

from 

schools_table a

join

students_table b

using (school_name)

where grade = '11th'

group by school_name)

using (school_name)

join 

(SELECT 

school_name,
avg(math_score) as '12th'

from 

schools_table a

join

students_table b

using (school_name)

where grade = '12th'

group by school_name)

using (school_name)
;

""",conn)

math_score_by_grade

,school_name,9th,10th,11th,12th
0,Farmer High School,82.016704,81.556962,81.430851,81.303191
1,Floyd High School,83.216981,82.911972,82.986434,83.002041
2,Greene High School,81.614241,80.940242,82.149020,82.293458
3,Hood High School,84.632911,83.254125,83.777228,82.537234
4,Long High School,83.763158,81.970060,84.052632,82.611465
5,Lopez High School,82.011494,81.554985,81.524057,81.791495
6,Patterson High School,81.156846,81.797034,81.576271,81.374348
7,Stewart High School,83.750000,84.240132,83.346614,83.685121
8,Thompson High School,83.377095,83.132791,83.831126,83.811728
9,Vargas High School,83.813918,83.087912,83.396414,84.068027


# Reading Score by Grade

In [10]:
# Read sqlite query results into a pandas DataFrame
# Read sqlite query results into a pandas DataFrame
reading_score_by_grade = pd.read_sql_query("""

select 

*

from 

(SELECT 

school_name,
avg(reading_score) as '9th'

from 

schools_table a

join

students_table b

using (school_name)

where grade = '9th'

group by school_name) c

join 

(SELECT 

school_name,
avg(reading_score) as '10th'

from 

schools_table a

join

students_table b

using (school_name)

where grade = '10th'

group by school_name)

using (school_name)

join 

(SELECT 

school_name,
avg(reading_score) as '11th'

from 

schools_table a

join

students_table b

using (school_name)

where grade = '11th'

group by school_name)

using (school_name)

join 

(SELECT 

school_name,
avg(reading_score) as '12th'

from 

schools_table a

join

students_table b

using (school_name)

where grade = '12th'

group by school_name)

using (school_name)
;

""",conn)

reading_score_by_grade

,school_name,9th,10th,11th,12th
0,Farmer High School,77.700445,76.520935,76.553191,77.344415
1,Floyd High School,93.933962,93.859155,93.955426,94.136735
2,Greene High School,77.118936,76.895613,76.508824,76.611215
3,Hood High School,94.320675,94.099010,94.108911,93.702128
4,Long High School,93.789474,93.532934,94.032895,93.910828
5,Lopez High School,76.580982,77.644399,77.697009,75.367627
6,Patterson High School,76.660581,76.145980,76.820975,77.116788
7,Stewart High School,94.186813,94.171053,93.896414,94.176471
8,Thompson High School,94.136872,94.252033,93.930464,94.108025
9,Vargas High School,93.922844,93.943681,94.009960,93.986395


# Scores by School Spending

In [11]:
# Read sqlite query results into a pandas DataFrame
score_by_school_spending = pd.read_sql_query("""

SELECT 

'Less than $1,000,000' as Spending_Ranges_Per_Student,
avg(c.Average_Math_Score) as Average_Math_Score,
avg(c.Average_Reading_Score) as Average_Reading_Score,
avg(c.Average_Overall_Score) as Average_Overall_Score,
avg((pass_count*100 / all_count) ||'%') as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_type,
sum(budget) as Total_Budget,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)

where Per_Student_Budget < 1000000

union

SELECT 

'Between $1,000,000 - $2,000,000' as Spending_Ranges_Per_Student,
avg(c.Average_Math_Score) as Average_Math_Score,
avg(c.Average_Reading_Score) as Average_Reading_Score,
avg(c.Average_Overall_Score) as Average_Overall_Score,
avg((pass_count*100 / all_count) ||'%') as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_type,
sum(budget) as Total_Budget,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)

where Per_Student_Budget between 1000000 and 2000000

union

SELECT 

'Greater than $2,000,000' as Spending_Ranges_Per_Student,
avg(c.Average_Math_Score) as Average_Math_Score,
avg(c.Average_Reading_Score) as Average_Reading_Score,
avg(c.Average_Overall_Score) as Average_Overall_Score,
avg((pass_count*100 / all_count) ||'%') as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_type,
sum(budget) as Total_Budget,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)

where Per_Student_Budget > 2000000
;

""",conn)

score_by_school_spending

,Spending_Ranges_Per_Student,Average_Math_Score,Average_Reading_Score,Average_Overall_Score,Overall_Passing_Rate
0,"Between $1,000,000 - $2,000,000",82.673301,88.162054,85.417678,76.333333
1,"Greater than $2,000,000",81.624198,76.836168,79.230183,51.000000
2,"Less than $1,000,000",83.483141,94.030815,88.756978,90.000000


# Scores by School Size

In [12]:
# Read sqlite query results into a pandas DataFrame
score_by_school_size = pd.read_sql_query("""

SELECT 

'Small (<1000)' as School_Size,
avg(c.Average_Math_Score) as Average_Math_Score,
avg(c.Average_Reading_Score) as Average_Reading_Score,
avg(c.Average_Overall_Score) as Average_Overall_Score,
avg((pass_count*100 / all_count) ||'%') as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_type,
sum(budget) as Total_Budget,
count(student_id) as Total_Student,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)

where Total_Student < 1000

union

SELECT 

'Medium (1000-2000)' as School_Size,
avg(c.Average_Math_Score) as Average_Math_Score,
avg(c.Average_Reading_Score) as Average_Reading_Score,
avg(c.Average_Overall_Score) as Average_Overall_Score,
avg((pass_count*100 / all_count) ||'%') as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_type,
sum(budget) as Total_Budget,
count(student_id) as Total_Student,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)

where Total_Student between 1000 and 2000

union

SELECT 

'Large (2000-5000)' as School_Size,
avg(c.Average_Math_Score) as Average_Math_Score,
avg(c.Average_Reading_Score) as Average_Reading_Score,
avg(c.Average_Overall_Score) as Average_Overall_Score,
avg((pass_count*100 / all_count) ||'%') as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_type,
sum(budget) as Total_Budget,
count(student_id) as Total_Student,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)

where Total_Student > 2000
;

""",conn)

score_by_school_size

,School_Size,Average_Math_Score,Average_Reading_Score,Average_Overall_Score,Overall_Passing_Rate
0,Large (2000-5000),82.073814,81.690119,81.881966,61.857143
1,Medium (1000-2000),83.644949,94.117666,88.881307,91.000000
2,Small (<1000),83.321332,93.943964,88.632648,89.000000


# Scores by School Type

In [13]:
# Read sqlite query results into a pandas DataFrame
score_by_school_type = pd.read_sql_query("""

SELECT 

'Charter' as School_Type,
avg(c.Average_Math_Score) as Average_Math_Score,
avg(c.Average_Reading_Score) as Average_Reading_Score,
avg(c.Average_Overall_Score) as Average_Overall_Score,
avg((pass_count*100 / all_count) ||'%') as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_type,
sum(budget) as Total_Budget,
count(student_id) as Total_Student,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)

where School_Type = 'Charter'

union

SELECT 

'District' as School_Type,
avg(c.Average_Math_Score) as Average_Math_Score,
avg(c.Average_Reading_Score) as Average_Reading_Score,
avg(c.Average_Overall_Score) as Average_Overall_Score,
avg((pass_count*100 / all_count) ||'%') as Overall_Passing_Rate

from 

(SELECT count(b.student_name) as all_count,
a.School_name,
type as School_type,
sum(budget) as Total_Budget,
count(student_id) as Total_Student,
sum(budget)/count(student_id) as Per_Student_Budget,
avg(math_score) as Average_Math_Score,
avg(reading_score) as Average_Reading_Score,
(avg(math_score) + avg(reading_score))/2 as Average_Overall_Score

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

group by a.School_name) c

join 

(SELECT count(b.student_name) as pass_count,
a.School_name

FROM schools_table a
inner join  students_table b
on a.school_name = b.school_name

where reading_score > 70
and math_score > 70

group by a.School_name
order by a.School_name) d

using (School_name)

where School_Type = 'District'
;

""",conn)

score_by_school_type

,School_Type,Average_Math_Score,Average_Reading_Score,Average_Overall_Score,Overall_Passing_Rate
0,Charter,83.422841,94.008532,88.715687,90.0
1,District,81.582442,76.780581,79.181511,50.6
